In [1]:
include("./ROIontology.jl")


ROIexp = load("./Data/ROIexp_FR.jld2", "ROIexp");
lfp = ROIexp[:lfp_data]
events = ROIexp[:behavioral_events]
trials = ROIexp[:trials]
sessions = ROIexp[:sessions]

include("./Links.jl")

WD = filter(x -> window_data_event(window_data(x)) ∈ dom(event_trial), collect(dom(window_data)))
WC = filter(x -> window_data_event(x) ∈ dom(event_trial), collect(codom(window_data)))

# WD = filter(x -> window_data_event(window_data(x)), collect(dom(window_data)))
# WC = filter(x -> window_data_event(x), collect(codom(window_data)))

1190-element Vector{Vector{Float64}}:
 [-0.09331660337249974, -0.09819589147917425, -0.09701250054912028, -0.10634067558245565, -0.1021405875302294, -0.09508570875457648, -0.10222455186734672, -0.09281840330007164, -0.04793675328783312, -0.057133355035141796  …  0.012000970506541482, 0.025241273428468974, 0.04294571134049535, 0.05226133557450283, 0.05718283311744984, 0.02164778581407128, -0.014923623848214692, 0.047510283889903915, 0.02752404050682769, 0.006529732011523056]
 [-0.13315295931214435, -0.13665682755355368, -0.11553105897029754, 0.01217419769535267, 0.12392939464261832, 0.16810780588327812, 0.13513692541079475, 0.15562960041545024, 0.14371666758303217, 0.1255655804435772  …  0.10221117842751364, 0.11151999093854242, 0.20395167324128263, 0.33968211693731504, 0.3435216408069801, 0.17291465716694754, -0.005134161633323996, -0.1969365304727736, -0.12453580032056083, -0.1391508127866966]
 [-0.01837882182201333, -0.05117392840454414, -0.05748236701138727, -0.04127012147527096, -0

In [12]:

using FourierAnalysis

# WT = event_trial∘window_data_event
# using JLD2 
# @save "TempData/event_lfp_windows_1_1.jld2"  WC
#split events into pre-peri-post with 512 sample pre and post windows
#events shorter than 512 are expanded
# wd_pre = GMap(x -> x[498:1010], WC)
wd_peri = GMap(x -> length(x) <= 2532 ? x[round(Int, length(x) / 2)-256:round(Int, length(x) / 2)+256] : x[1010:end-1010], WC)
# wd_peri = GMap(x -> x, WC)

# wd_post = GMap(x -> x[end-1010:end-498], WC)


# function isartifact(x::DataWindow)
#     if x.data.region==MOB
#         thresh=.75
#     else
#         thresh=.6
#     end
#     # if any(window_data(x).≥thresh)
#     if sum((window_data(x) .≥ thresh)) > (length(window_data(x)) * .01)
#         return true
#     else
#         return false
#     end
# end
# wd_peri=filter(!isartifact, win)

# #Freq Dom
# # pre_spec = GMap(x -> spectra(x, 256, 512), wd_pre.(WC))
peri_spec = GMap(x -> spectra(x, 256, 512), wd_peri.(WC))
# # post_spec = GMap(x -> spectra(x, 256, 512), wd_post.(WC)) 

using DataFrames, DataFramesMeta

event_spectra = DataFrame()

event_spectra.rat = map(x -> window_lfp(inv(window_data)(x)).rat.name, WC)
event_spectra.region = map(x -> window_data_region(x).name, WC)
event_spectra.behavior_type = map(x -> window_data_event(x).behavior.name, WC)
event_spectra.trial_type = map(x -> event_trial(window_data_event(x)).condition.name, WC)
event_spectra.agent_type = map(x -> agenttype(event_trial(window_data_event(x)).condition), WC)
specs = map(x -> spectra(x, 256, 512), WC)
event_spectra.respiratory = map(x -> mean(x, (3, 12) .* 256 ./ 1010.1), specs)
event_spectra.theta = map(x -> mean(x, (5, 10) .* 256 ./ 1010.1), specs)
event_spectra.beta = map(x -> mean(x, (15, 35) .* 256 ./ 1010.1), specs)
event_spectra.gamma_low = map(x -> mean(x, (50, 59) .* 256 ./ 1010.1), specs)
event_spectra.gamma_high = map(x -> mean(x, (70, 100) .* 256 ./ 1010.1), specs)


event_spectra = @subset(event_spectra, :trial_type .== "Free Roam", :region .∈ Ref(["MOB", "Amygdala", "Ca2"]))


,rat,region,behavior_type,trial_type,agent_type,respiratory,theta
,String7,String,String15,String,DataType,Float64,Float64
1,RRSD28,Ca2,Rearing,Free Roam,Robot,0.00480679,0.00552241
2,RRSD28,MOB,Immobility,Free Roam,Robot,0.0185517,0.015892
3,RRSD17,MOB,Rearing,Free Roam,Object,0.007176,0.00876942
4,RRSD17,Amygdala,Immobility,Free Roam,Object,0.00154076,0.00167873
5,RRSD17,Ca2,Immobility,Free Roam,Robot,0.00195673,0.00247963
6,RRSD17,MOB,Immobility,Free Roam,Rat,0.0124936,0.0149751
7,RRSD17,Amygdala,Grooming,Free Roam,Rat,0.00189254,0.00177685
8,RRSD28,Ca2,Immobility,Free Roam,Robot,0.00457848,0.0039664
9,RRSD17,Amygdala,Grooming,Free Roam,Rat,0.00151663,0.00172818


In [14]:
ratidx = rand(findall(x -> x == Rat, collect(event_spectra.agent_type)), 50)
robotidx = rand(findall(x -> x == Robot, collect(event_spectra.agent_type)), 50
objectidx = findall(x -> x == Object, collect(event_spectra.agent_type))

esnull = copy(event_spectra)
esnull.behavior_type = fill("null", length(esnull.behavior_type))



330-element Vector{Int64}:
    3
    4
   15
   17
   27
   31
   33
   35
   38
   39
   41
   44
   47
    ⋮
  993
  995
  998
 1003
 1005
 1015
 1023
 1039
 1043
 1047
 1048
 1054

In [ ]:
using MixedModels, ProgressMeter
ProgressMeter.ijulia_behavior(:append)

1) It is also hypothesized that hippocampal theta oscillation will increase in amplitude during rearing, due to theta being an exploration related sensorimotor rhythm. This difference should also be present in amygdala (though may be due to volume conduction).

In [ ]:
hyposub=@subset(event_spectra, :behavior_type .∈ Ref(["null","Rear"]), :region.=="Ca2")
fm=@formula(theta ~ 1 + behavior_type*1 + (1|rat))
fm1 = fit(MixedModel,fm, hyposub, contrasts = Dict(:behavior_type => DummyCoding(;base="null")))

In [ ]:
hyposub=@subset(event_spectra, :behavior_type .∈ Ref(["null","Rear"]), :region.=="Amygdala")
fm=@formula(theta ~ 1 + behavior_type*1 + (1|rat))
fm1 = fit(MixedModel,fm, hyposub, contrasts = Dict(:behavior_type => DummyCoding(;base="null")))

2. We expect that beta rhythms will be present when the animal is rearing, and not present during immobility. 

In [ ]:
hyposub=@subset(event_spectra, :behavior_type .∈ Ref([ "null","Rear"]))
fm=@formula(beta ~0+region+ behavior_type&region + zerocorr(0+region|rat))
fm1 = fit(MixedModel,fm, hyposub, contrasts = Dict(:behavior_type => DummyCoding(;base="null")))

3. It is hypothesized that theta will show increased amplitude during behaviors associated with exploration like rearing, when compared with regulatory behaviors like grooming and immobility. This means that rearing will be significant different from immobility/grooming in both respiratory and theta rhythms in MOB, CA2, and Amyg. 

In [ ]:
hyposub=@subset(event_spectra, :behavior_type .∈ Ref(["Immobility","Groom","Rear"]))
function fun(btype)
    if btype .== "Immobility"
        return "Immobility & Groom"
    elseif btype .== "Groom"
        return "Immobility & Groom"
    else
        return btype
    end
end
@transform!(hyposub, @byrow :combined_behavior = fun(:behavior_type))
fm=@formula(theta ~0+region+ combined_behavior&region + zerocorr(0+region|rat))
fm1 = fit(MixedModel,fm, hyposub, contrasts = Dict(:combined_behavior => DummyCoding(;base="Immobility & Groom")))

In [ ]:
fm=@formula(respiratory ~0+region+ combined_behavior&region + zerocorr(0+region|rat))
fm1 = fit(MixedModel,fm, hyposub, contrasts = Dict(:combined_behavior => DummyCoding(;base="Immobility & Groom")))

4. It is hypothesized that robot will elicit a higher low-gamma rhythm in MOB for the robot condition when compared with the object and rat conditions for immobility and grooming behaviors. 


In [ ]:
hyposub = @subset(event_spectra, :agent_type .∈ Ref([Rat, Robot, Object]), :region .== "MOB", :behavior_type .∈ Ref(["Immobility", "Groom", "Rear", "null"]))
function afun(atype)
    if atype .== Rat
        return "Rat & Object"
    elseif atype .== Object
        return "Rat & Object"
    else
        return "Robot"
    end
end
@transform!(hyposub, @byrow :combined_agent = afun(:agent_type))
fm = @formula(gamma_low ~ 0 + behavior_type + combined_agent & behavior_type + (0 + 1 | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:combined_agent => DummyCoding(; base = "Rat & Object")))

In [ ]:
hyposub = @subset(event_spectra, :agent_type .∈ Ref([Rat, Robot, Object]), :region .== "MOB", :behavior_type .∈ Ref(["Immobility", "Groom", "Rear", "null"]))
function afun(atype)
    if atype .== Rat
        return "Rat & Object"
    elseif atype .== Object
        return "Rat & Object"
    else
        return "Robot"
    end
end
@transform!(hyposub, @byrow :combined_agent = afun(:agent_type))
fm = @formula(gamma_low ~ 0 + behavior_type + combined_agent & behavior_type + (0 + 1 | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:combined_agent => DummyCoding(; base = "Rat & Object")))

5. High gamma amplitude should be highest for the rat condition in the MOB. The robot and object should have similar high gamma. But instead, it appears that the robot and rat high gamma is larger than the object, and there is no discernable difference between rat and robot high gamma in MOB. 

In [ ]:
hyposub = @subset(event_spectra, :agent_type .∈ Ref([Rat, Robot, Object]), :region .== "MOB", :behavior_type .∈ Ref(["Immobility", "Groom", "Rear", "null"]))
function afun(atype)
    if atype .== Rat
        return "Rat"
    elseif atype .== Object
        return "Object"
    else
        return "Robot"
    end
end
@transform!(hyposub, @byrow :agent_type = afun(:agent_type))
fm = @formula(gamma_high ~ 0 + behavior_type + agent_type & behavior_type + (0 + 1 | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:agent_type => DummyCoding(; base = "Rat")))


In [ ]:
hyposub = @subset(event_spectra, :agent_type .∈ Ref([Rat, Robot, Object]), :region .== "MOB", :behavior_type .∈ Ref(["Immobility", "Groom", "Rear", "null"]))
function afun(atype)
    if atype .== Rat
        return "Rat"
    elseif atype .== Object
        return "Object"
    else
        return "Robot"
    end
end
@transform!(hyposub, @byrow :agent_type = afun(:agent_type))
fm = @formula(gamma_low ~ 0 + behavior_type + agent_type & behavior_type + (0 + 1 | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:agent_type => DummyCoding(; base = "Robot")))


6. It is hypothesized that the amygdala beta rhythm shows a difference between grooming/immobility and rearing. It would be expected that immobility shows the least beta, with grooming showing some due to arm movements, and rearing showing the most. 

In [ ]:
hyposub = @subset(event_spectra, :behavior_type .∈ Ref(["Immobility", "Groom", "Rear"]), :region .== "Amygdala")
function fun(btype)
    if btype .== "Immobility"
        return "Immobility & Groom"
    elseif btype .== "Groom"
        return "Immobility & Groom"
    else
        return btype
    end
end
@transform!(hyposub, @byrow :combined_behavior = fun(:behavior_type))
fm = @formula(beta ~ 1 + combined_behavior * 1 + (1 | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:combined_behavior => DummyCoding(; base = "Immobility & Groom")))

7.  For regulatory behaviors (grooming/immobility) there should be a shift to lower frequencies within the respiratory range and more low gamma, while exploratory behaviors should shift to theta frequency and high gamma (rearing, approach/retreat). 

In [ ]:
hyposub = @subset(event_spectra, :behavior_type .∈ Ref(["Immobility", "Groom", "null"]))
function fun(btype)
    if btype .== "Immobility"
        return "Immobility & Groom"
    elseif btype .== "Groom"
        return "Immobility & Groom"
    else
        return btype
    end
end
@transform!(hyposub, @byrow :combined_behavior = fun(:behavior_type))
fm = @formula(theta ~ 0 + region + combined_behavior & region + zerocorr(0 + region | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:combined_behavior => DummyCoding(; base = "null")))

In [ ]:
hyposub = @subset(event_spectra, :behavior_type .∈ Ref(["Immobility", "Groom", "null"]))
function fun(btype)
    if btype .== "Immobility"
        return "Immobility & Groom"
    elseif btype .== "Groom"
        return "Immobility & Groom"
    else
        return btype
    end
end
@transform!(hyposub, @byrow :combined_behavior = fun(:behavior_type))
fm = @formula(gamma_high ~ 0 + region + combined_behavior & region + zerocorr(0 + region | rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:combined_behavior => DummyCoding(; base = "null")))

In [ ]:
hyposub = @subset(event_spectra, :agent_type .∈ Ref([Rat, Robot, Object]), :behavior_type .∈ Ref(["Immobility"]))
function afun(atype)
    if atype .== Rat
        return "Rat"
    elseif atype .== Object
        return "Object"
    else
        return "Robot"
    end
end
@transform!(hyposub, @byrow :agent_type = afun(:agent_type))
fm=@formula(theta ~0+region+ agent_type&region + zerocorr(0+region|rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:agent_type => DummyCoding(; base = "Rat")))

In [ ]:
hyposub = @subset(event_spectra, :agent_type .∈ Ref([Rat, Robot, Object]), :behavior_type .∈ Ref(["Rear"]))
function afun(atype)
    if atype .== Rat
        return "Rat"
    elseif atype .== Object
        return "Object"
    else
        return "Robot"
    end
end
@transform!(hyposub, @byrow :agent_type = afun(:agent_type))
fm=@formula(theta ~0+region+ agent_type&region + zerocorr(0+region|rat))
fm1 = fit(MixedModel, fm, hyposub, contrasts = Dict(:agent_type => DummyCoding(; base = "Rat")))